<a href="https://colab.research.google.com/github/GerardoMunoz/PresentacionesAlgebraLineal/blob/main/PS_angulo_word_vect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En la página https://matematicaj.blogspot.com/2020/03/Test-de-analogias-verbales-resuelto-con-claves-y-respuestas-pre-universidad-pdf.html se presentan algunas analogías de palabras. En el primer caso se pregunta soldado es a ejército como:
1. Ratón a gato,
2. Llave a llavero,
3. Paradoja a contradicción,
4. Pulga a perro,
5. Tornillo a máquina.

Usaremos el álgebra lineal para resolver el problema. A principio de semestre vimos como  algoritmos (como bolsa de palabras, Word2vec, Gensim, GloVe) aplicado a un conjunto de documentos (corpus), [incrustan las palabras](https://unipython.com/como-desarrollar-embeddings-incrustaciones-de-palabras-con-gensim/) <!--(https://es.wikipedia.org/wiki/Word_embedding)--> en $\mathbb{R}^n$.  

Algunas de estas incrustaciones tienen propiedades semánticas, es decir que algunas operaciones vectoriales son congruentes con el significado de las palabras. Como en el famoso ejemplo  Rey - Hombre + Mujer -> Reina. 

Para este ejercicio vamos a utilizar la librería `spacy` con una base de datos de noticias (news) pequeña (sm) en español (es) llamada `es_core_news_sm`. 

* Lo primero es obtener el vector $\vec{u}_0$  para la palabra soldado y el vector $\vec{v}_0$ para la palabra ejército. Los vectores no están ni en $\mathbb{R}^2$ ni en $\mathbb{R}^3$ sino en $\mathbb{R}^{96}$

* Luego se obtiene el vector  $\vec{w}_0 =  \vec{u}_0 - \vec{v}_0$ que va desde soldado hasta ejército.

* De manera similar se obtiene el vector para la primera repuesta. Es decir, el vector $\vec{w}_1$ que va de raton a gato.  Finalmente se calcula el coseno del ángulo $\cos(\alpha_1)$ entre el vector  $\vec{w}_0$ y $\vec{w}_1$.

* Se repite los mismo para cada par de palabras, obteniendo los vectores $\vec{w}_2$, $\vec{w}_3$, $\vec{w}_4$, $\vec{w}_5$. Posteriormente los cosenos  $\cos(\alpha_2)$, $\cos(\alpha_3)$, $\cos(\alpha_4)$, $\cos(\alpha_5)$, entre los vectores  $\vec{w}_0$ y el respectivo $\vec{w}_i$.

*  Finalmente se selecciona la respuesta con el mayor coseno del ángulo.

Lo primero es descargar la librería en español "es_core_news_sm". Esto sólo hay que realizarlo una vez se inicia. 



In [ ]:
!python -m spacy download es_core_news_sm

En Colab puede 'Reinicar el entorno de ejecución' despues de descargar la librería 

Lo primero es llamar la libreria spacy. 

In [ ]:
import spacy

Luego sigue cargar los datos del español que se descargaron previamente. En caso de no haber reiniciado el entorno de ejecución puede que se presente un error.

In [ ]:
spacy_es = spacy.load("es_core_news_sm")

Ahora si comenzamos a obtener los vectores de cada palabra. Es importante que la palabra se escriba con tilde si las tiene. 

In [ ]:
soldado=spacy_es("soldado").vector
soldado

array([ 1.6616583 ,  2.2189255 ,  0.34981376,  1.6703401 ,  2.0867515 ,
        1.3361075 , -0.16638641,  1.7989249 , -0.3785099 , -0.9531485 ,
       -4.787784  ,  0.5524312 , -0.27699035, -4.69687   , -1.3153002 ,
       -2.1852336 ,  0.08720505, -1.7302126 ,  2.4323547 ,  1.7860228 ,
       -0.19702435, -2.0001698 , -1.3060259 , -2.4178617 , -0.43159366,
        1.8175896 , -0.21075806, -0.9220848 , -1.0492164 , -0.6611053 ,
       -1.0855255 , -0.8291515 ,  0.27188474,  0.56854105,  0.51144755,
        1.1975454 , -0.676649  ,  1.793778  , -2.2901528 ,  1.2468784 ,
       -1.7368109 ,  1.179375  ,  0.1729405 ,  2.7017672 , -0.74960184,
        4.0208335 ,  1.1929208 ,  1.8065389 ,  0.2725471 , -1.377909  ,
        1.8582463 ,  0.10081023, -4.811147  ,  3.0806613 ,  2.0560303 ,
       -1.5004798 ,  3.8858168 , -0.33191317,  0.515605  ,  0.6267386 ,
       -0.66777545,  3.5780149 ,  1.2418798 ,  1.9644146 , -3.0513065 ,
       -0.909126  , -3.0529184 ,  2.6292071 ,  1.071341  , -2.80

Observe que cada palabra es un vector de 96 elementos

In [ ]:
len(soldado)

96

In [ ]:
ejercito=spacy_es("ejército").vector
ejercito

array([ 1.5633483e+00,  1.4476252e-01,  1.8350602e+00,  1.1558589e-01,
        2.3402157e+00, -1.4280640e+00,  3.0933189e+00,  1.2887498e+00,
       -1.9117045e-01, -4.9598169e+00, -1.2467630e+00, -2.7395284e+00,
       -2.5109317e+00, -2.6089320e+00, -1.6932045e+00, -9.9181211e-01,
       -1.3153166e-02,  2.4071431e+00, -4.3894949e+00,  2.0477021e+00,
        1.2002409e+00, -2.5596815e-01, -4.4633490e-01,  2.9005063e-01,
        5.9353590e-01,  1.2809510e+00,  9.2161834e-01, -9.1395527e-01,
       -3.1797333e+00, -2.9567182e+00, -2.7443204e+00, -1.4416915e+00,
        9.0844846e-01, -1.4594206e+00,  5.6294864e-01,  2.2365236e+00,
       -9.3573844e-01,  1.3580439e+00, -3.6455970e+00, -1.7200713e+00,
       -1.0916401e+00,  3.9986399e-01,  1.9496071e+00, -3.0031584e-02,
        1.4883491e+00, -2.2285834e-01,  4.9609989e-01,  7.1523711e-04,
       -6.2915820e-01, -1.3156153e+00,  1.1743956e+00,  8.6497825e-01,
        5.1256812e-01, -3.1079383e+00,  1.8793962e+00, -1.5822583e+00,
      

In [ ]:
soldado_ejercito = soldado - ejercito
soldado_ejercito

array([ 0.09830999,  2.074163  , -1.4852464 ,  1.5547541 , -0.25346422,
        2.7641716 , -3.2597053 ,  0.5101751 , -0.18733945,  4.0066686 ,
       -3.541021  ,  3.2919598 ,  2.2339413 , -2.0879378 ,  0.3779043 ,
       -1.1934215 ,  0.10035822, -4.137356  ,  6.82185   , -0.2616793 ,
       -1.3972652 , -1.7442017 , -0.85969096, -2.7079124 , -1.0251296 ,
        0.5366386 , -1.1323764 , -0.00812954,  2.130517  ,  2.2956128 ,
        1.6587949 ,  0.61254   , -0.6365637 ,  2.0279617 , -0.0515011 ,
       -1.0389782 ,  0.25908947,  0.43573403,  1.3554442 ,  2.9669497 ,
       -0.6451708 ,  0.7795111 , -1.7766666 ,  2.731799  , -2.2379508 ,
        4.243692  ,  0.6968209 ,  1.8058237 ,  0.90170527, -0.06229365,
        0.68385077, -0.764168  , -5.323715  ,  6.1885996 ,  0.17663407,
        0.08177853,  4.7113695 ,  0.7269333 , -0.38351035, -0.63434726,
       -0.15181297,  4.0158362 , -1.1249925 ,  2.434235  , -2.8239462 ,
       -0.60974663, -0.33697534,  1.4745375 ,  1.3492615 , -1.03

In [ ]:
raton = spacy_es("ratón").vector
gato = spacy_es("gato").vector
raton_gato = raton - gato
raton_gato

array([-1.265033  ,  1.3099129 ,  1.1179032 , -1.5713887 ,  1.1892524 ,
       -1.3013593 ,  0.5610335 , -1.5968785 , -2.3235073 , -2.8649027 ,
        1.8099622 , -5.303787  , -2.9253464 ,  1.0467217 , -1.9469875 ,
       -2.0728583 ,  2.5955985 ,  1.9755002 , -4.0862637 ,  1.0056701 ,
       -1.3438596 , -1.7438902 ,  0.3450884 ,  0.8864556 ,  4.3622103 ,
        2.4505277 , -0.22487482, -0.5231006 ,  2.2676368 , -0.9214817 ,
       -0.458297  , -3.4453602 ,  1.3129684 ,  1.7771535 , -2.6731868 ,
        1.9201785 ,  5.252463  ,  4.4567494 ,  1.0975027 , -1.6185219 ,
       -0.20419598,  0.41635224,  4.443267  , -2.580287  ,  3.183191  ,
       -1.2470889 ,  2.9683027 ,  0.45370248, -3.311287  , -0.8524591 ,
        0.8026655 , -1.5846772 , -1.7272313 , -3.6841402 ,  2.6649466 ,
       -0.9283836 , -1.5784795 , -1.1975842 ,  2.6008167 , -2.9763076 ,
       -0.62568986,  1.7695354 ,  2.0158207 ,  1.6911104 ,  0.82638323,
        2.3242762 , -1.6823988 , -2.215893  , -2.1800365 , -1.35

In [ ]:
import numpy as np
# Los vectores obtenidos son arreglos de numpy
# El producto punto entre dos vectores se puede realizar con el método `dot`
# La norma de un vector se realiza con la función `np.linalg.norm`

In [ ]:
cos_preg_rta1 = soldado_ejercito.dot(raton_gato) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(raton_gato))
cos_preg_rta1

-0.3152439

In [ ]:
llave = spacy_es("llave").vector
llavero = spacy_es("llavero").vector
llave_llavero = llave - llavero

cos_preg_rta2 = soldado_ejercito.dot(llave_llavero) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(llave_llavero))
cos_preg_rta2

0.23535857

In [ ]:
paradoja = spacy_es("paradoja").vector
contradiccion = spacy_es("contradicción").vector
paradoja_contradiccion = paradoja - contradiccion

cos_preg_rta3 = soldado_ejercito.dot(paradoja_contradiccion) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(paradoja_contradiccion))
cos_preg_rta3

-0.02385487

In [ ]:
pulga = spacy_es("pulga").vector
perro = spacy_es("perro").vector
pulga_perro = pulga - perro

cos_preg_rta4 = soldado_ejercito.dot(pulga_perro) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(pulga_perro))
cos_preg_rta4

0.21286036

In [ ]:
tornillo = spacy_es("tornillo").vector
maquina = spacy_es("máquina").vector
tornillo_maquina = tornillo - maquina

cos_preg_rta5 = soldado_ejercito.dot(tornillo_maquina) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(tornillo_maquina))
cos_preg_rta5

-0.03068008

coseno del ángulo con soldado ejercito    
| palabra1 | palabra2    |        |
|----------|-------------|---------------|
|ratón     |gato         |-0.3152439     |
|llave     |llavero      | 0.23535857    |
|paradoja  |contradicción|-0.02385487   |
|pulga     |perro        | 0.21286036   |
|tornillo  |máquina      |-0.03068008    |

La mejor respuesta es la que el coseno del ángulo sea mayor. En este caso sería llave-llavero, que efectivamente corresponde a la respuesta correcta. Ya que al igual que soldado-ejérciro, ambas tienen una relación de elemento - conjunto. 


En el siguiente listado está el ejercicio que le corresponde de la página https://matematicaj.blogspot.com/2020/03/Test-de-analogias-verbales-resuelto-con-claves-y-respuestas-pre-universidad-pdf.html

|	Ejercicio	|	Código	|
|	----	|	-------	|
|	1	|	Ejemplo	|
|	2	|	20172005165	|
|	3	|	20181005165	|
|	4	|	20182005001	|
|	5	|	20182005130	|
|	6	|	20191005103	|
|	7	|	20191005154	|
|	8	|	20191005158	|
|	9	|	20192005014	|
|	10	|	20192005029	|
|	11	|	20192005041	|
|	12	|	20192005110	|
|	13	|	20192005144	|
|	14	|	20201005013	|
|	15	|	20201005027	|
|	16	|	20201005046	|
|	17	|	20201005101	|
|	18	|	20201005131	|
|	19	|	20201005134	|
|	20	|	20201005169	|
|	21	|	20201015147	|
|	22	|	20201015176	|
|	23	|	20202005044	|
|	24	|	20202005047	|
|	25	|	20202005050	|
|	26	|	20202005083	|
|	27	|	20202005095	|
|	28	|	20202005096	|
|	29	|	20202005098	|
|	30	|	20202005101	|
|	31	|	20202005111	|
|	32	|	20202005112	|
|	33	|	20202005115	|
|	34	|	20202005118	|
|	35	|	20202005146	|
|	36	|	20202015006	|
|	37	|	20202015013	|
|	38	|	20202015015	|
|	39	|	20202015019	|
|	40	|	20202015028	|
|	41	|	20202015134	|

Llene los datos solicitados en el respectivo formulario

